# Лабораторная работа 2 # 
## Студент: Девяткина Д.В. 
## Группа: М8О-304Б


### Постановка задачи
1. Требуется реализовать класс на выбранном языке программирования, который реализует один из алгоритмов машинного обучения. Обязательным является наличия в классе двух методов fit, predict. 
2. Необходимо проверить работу вашего алгоритма на ваших данных (на таблице и на текстовых данных), произведя необходимую подготовку данных. 
3. Также необходимо реализовать алгоритм полиномиальной регрессии, для предсказания значений в таблице. 
4. Сравнить результаты с стандартной реализацией sklearn, определить в чем сходство и различие ваших алгоритмов. 
5. Замерить время работы алгоритмов.

### Алгоритм
Наивный байесовский классификатор
 
4 % 6 + 1


### Описание алгоритма

Наивный Байесовский классификатор основан на теореме Байеса и является "наивным", потому что мы делаем допущение о том, что признаки независимы.

В своей работе я реализовала 2 модели байесовского классификатора. 
1. *Gaussian* (нормальное распределение). Модель данного типа используется в случае непрерывных признаков и предполагает, что значения признаков имеют нормальное распределение.
Это распределение удобно использовать с численными данными, поэтому я его приложила к численному датасету.
2. *Multinomial* (мультиномиальное распределение). Используется в случае дискретных признаков. Например, в задаче классификации текстов признаки могут показывать, сколько раз каждое слово встречается в данном тексте.
Эта модель приложена к корпусу документов. 


## Реализация Мультиномиальной модели

Была использована формула:

### Подготовка данных текстового датасета

Выбранный [корпус](https://github.com/kavgan/OpinRank/blob/master/OpinRankDatasetWithJudgments.zip) документов - отзывы на отели в 3 городах: Лондон, Дубаи, Лас-Вегас

*Предварительно разделяю на класс "Дубаи" и "Лондон".*

Количество отзывов на Дубаи: 236

Выбранно количество отзывов на Лондон: 250 (так как размер словаря после всей обработки при, например, 690 файлах, переваливает за 100к слов, и выходит MemoryError)

Задача: Определение к какому классу (городу) относятся отзывы.


## Алгоритм
Вероятность считается по следующей формуле: 

![основная формула](https://cdn-images-1.medium.com/max/1000/1*4XdLoLXubDsN9Jix_OdJVA.gif)

где pi(j) - вероятность документов класса j среди всех документов

![т](https://cdn-images-1.medium.com/max/1000/1*QbeBoi9mCJPLKBG6YlW9Sg.gif)

IDF (Inverse Document Frequency), однако в последствии я убрала её из расчета, так как время работы алгоритма с ней увеличилось в значительной степени. 

![лаплас](https://cdn-images-1.medium.com/max/1000/1*L0RGaXZGrsAxkkFoINabUw.png)

Сглаживание Лапласа с малым параметром альфа и V - размер словаря.


In [1]:
#импортируем нужные библиотеки
import pandas as pd
import numpy as np
import os
import re
import nltk
import time
import math
import random

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB

from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

In [2]:
#создаем датафрейм с названием отеля, текстом отзывов, городом (классом)
dir1 = 'dubai/'
dir3 = 'london/'
files1 = os.listdir(dir1)
files3 = os.listdir(dir3)
files3 = files3[0:260]

df = pd.DataFrame(columns = ['hotel', 'review'])

towns = [files1, files3]
dirr = [dir1, dir3]

i, j = 0, 0
for files in towns:
    for file in files:
        text = ''
        hotel = open(dirr[j]+file, 'r')
        for line in hotel: text += line
        df.loc[i] = [file, text]
        i += 1
    j += 1

town = []
for hotel in df['hotel']:
    if 'dubai' in hotel: town.append(0)
    else: town.append(1)

#получаем датафрейм со столбцами [отель, отзыв, город(0/1 или Дубаи/Лондон)]
df['town'] = town

#меняем все буквы на строчные
df['review'] = df.review.map(lambda x: x.lower()) 

#удаляем все цифры
df['review'] = df.review.str.replace('\d', '')

#удаляем пунктуацию
df['review'] = df.review.str.replace('[^\w\s]', '')

#токенизация
df['review'] = df['review'].apply(nltk.word_tokenize) 
df['review'].head()

0    [oct, good, overall, service, just, came, back...
1    [value, for, money, the, hotel, has, very, goo...
2    [jun, excellent, and, near, the, airport, i, h...
3    [nov, great, for, relaxing, stopovers, this, i...
4    [oct, information, about, alhijaz, heritage, m...
Name: review, dtype: object

In [3]:
#стемминг 
stemmer = SnowballStemmer("english", ignore_stopwords=True)
df['review'] = df['review'].apply(lambda x: [stemmer.stem(y) for y in x])

# перевод списка слов обратно в строку
df['review'] = df['review'].apply(lambda x: ' '.join(x))
df['review'].head()

#сконвертируем набор текстов в матрицу токенов
count_vect = CountVectorizer() 
counts = count_vect.fit_transform(df['review']).toarray()

print('Размер полученного словаря: ', len(counts[0]))

x_train, x_test, y_train, y_test = train_test_split(counts, df['town'].values, test_size = 0.4)
print('Разделено {0} документов на {1} обучающих и {2} тестовых'.format(len(counts),len(x_train),len(x_test)))

Размер полученного словаря:  79569
Разделено 496 документов на 297 обучающих и 199 тестовых


In [4]:
alpha = 0.001
class NBMultinomial:
    def __init__(self):
        self.stat = {} #{класс: (количество докуменов с таким классом, количество слов в этом классе, массив вхождений слов в этом классе)}
        self.words = 0 #количество слов в словаре 
        self.docs = 0 #количество документов в обучающей выборке
        
    def fit(self, counts, train):
        #получим классы и количество каждого класса
        classes, num_c = np.unique(train, return_counts=True)
        self.words = len(counts[0])
        self.docs = len(train)
        self.stat = {classes[i]: (num_c[i], 0, np.array([0] * self.words)) for i in range(len(classes))}
        #для каждого класса получим статистику
        for i in range(self.docs):
            stat_doc = self.stat[train[i]]
            self.stat[train[i]] = (stat_doc[0], stat_doc[1] + sum(counts[i]), stat_doc[2] + counts[i]) 

    def predict(self, test):
        predicted = []
        for i in range(len(test)):
            #выбор наибольшей вероятности и присваиваем соответствующий класс
            best_c, best_p = None, -math.inf
            for c in self.stat.keys():
                #вероятность класса среди всех документов
                P = math.log(self.stat[c][0] / self.docs)
                for j in range(self.words):
                    #чтобы не было делений на 0 проверка на нулевое количество вхождений слова
                    if test[i][j] != 0:
                        #Inverse Document Frequency (IDF) weight on each word
                        #ЗАМЕЧАНИЕ: видимо из-за того, что используется сумма строки на каждой итерации
                        #время работы опять возрасло и я выкинула это из формулы
                        #idf = math.log(sum(test[i]) / test[i][j])
                        #применяю сглаживание Лапласа
                        #P += test[i][j] * math.log(idf * (self.stat[c][2][j] + alpha) / (self.stat[c][1] + self.words + 1))
                        P += test[i][j] * math.log((self.stat[c][2][j] + alpha) / (self.stat[c][1] + self.words + 1))
                if P > best_p: 
                    best_p, best_c = P, c
            predicted.append(best_c)
        return predicted

In [14]:
start = time.time()

mnb = NBMultinomial()
mnb.fit(x_train, y_train)
y_pred = mnb.predict(x_test)

print('Неправильно предсказано: {} из {}\nТочность: {}'.format((y_test != y_pred).sum(), len(y_pred), accuracy_score(y_test, y_pred)))
print('Матрица ошибок:\n', confusion_matrix(y_test, y_pred))

finish = time.time()
print('Время работы собственного классификатора: {} секунд'.format(finish-start))

Неправильно предсказано: 1 из 199
Точность: 0.9949748743718593
Матрица ошибок:
 [[ 93   1]
 [  0 105]]
Время работы собственного классификатора: 31.976750135421753 секунд


## Получим те же результаты с помощью библиотеки sklearn

In [13]:
start = time.time()

mnb_sk = MultinomialNB()
mnb_sk.fit(x_train, y_train)
y_pred2 = mnb_sk.predict(x_test)

print('Неправильно предсказано: {} из {}\nТочность: {}'.format((y_test != y_pred2).sum(), len(y_pred2), accuracy_score(y_test, y_pred2)))
print('Матрица ошибок:\n', confusion_matrix(y_test, y_pred2))

finish = time.time()
print('Время работы классификатора sklearn: {} секунд'.format(finish-start))

Неправильно предсказано: 2 из 199
Точность: 0.9899497487437185
Матрица ошибок:
 [[ 92   2]
 [  0 105]]
Время работы классификатора sklearn: 6.638439893722534 секунд


# НБК для категориального датасета

*Выбранный [датасет](https://www.kaggle.com/nisargpatel/automobiles) - информация об автомобилях*

Выглядит он так:

In [5]:
ad.head()


,symboling,normalized_losses,make,fuel_type,aspiration,number_of_doors,body_style,drive_wheels,engine_location,wheel_base,...,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
0,3,168,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,168,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,168,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


## Реализация Гауссовской модели

Необходимо найти наиболее вероятный класс объекта x, Байесовский классификатор использует оценку апостериорного максимума (Maximum a posteriori estimation) для определения наиболее вероятного класса. т.е. максимум вероятности P(y=c|x). Вероятность признаков предполагается распреденной нормально:

![вер](https://i.postimg.cc/P5JCxW2g/image.png)

### Подготовка данных категориального датасета

Задача: предсказать количество дверей автомобиля в зависимости от типа кузова (кабриолет, хэтчбек, седан и т.д.), переднего/заднего/полного привода и есть ли турбонаддув. 

Как можно увидеть, они не занесены в численном виде, значит необходимо провести подготовку данных. Для этого, например, заменим в признаке видов привода fwd -> 0, rwd -> 1, 4wd -> 2. Проведем аналогичные операции с другими выбранными признаками.

## Код программы:

In [7]:
ad = pd.read_csv('Automobile.csv')

#перевожу категориальные признаки в численные
ad['numdoors_cleaned'] = np.where(ad['number_of_doors'] == 'two', 0, 1)
ad['salon_cleaned'] = np.where(ad['body_style'] == 'convertible', 0, 
                               np.where(ad['body_style'] == 'hardtop',1, 
                                        np.where(ad['body_style'] == 'hatchback',2, 
                                                 np.where(ad['body_style'] == 'sedan',3, 4))))
ad['wheels_cleaned'] = np.where(ad['drive_wheels'] == 'fwd', 0,
                                np.where(ad['drive_wheels'] == 'rwd',1, 2))
ad['aspiration_cleaned'] = np.where(ad['aspiration'] == 'std', 0, 1)

used_features = [
    'aspiration_cleaned',
    'salon_cleaned',
    'wheels_cleaned',
    'numdoors_cleaned']

#разбиение
X_train, X_test = train_test_split(ad, test_size=0.33)
print('Разделено {0} строк на {1} обучающих и {2} тестовых строк'.format(len(ad),len(X_train),len(X_test)))

#функция поиска медианы
def mean(nums):
    mean = sum(nums) / float(len(nums))
    return mean

#функция поиска стандартного отклонения 
def stdev(nums):
    variance = sum([pow(x - mean(nums), 2) for x in nums])/float(len(nums) - 1)
    return math.sqrt(variance)

#функция вероятности нормального распределения
def calcGauss(x, m, d):
    exponent = math.exp( -(math.pow(x - m, 2) / (2 * math.pow(d, 2))))
    return (1 / (math.sqrt(2 * math.pi) * d)) * exponent

#функция для предсказания класса для одного элемента(вектора) обучающей выборки
def get_predict(stat, test_vec):
    #вероятность каждого класса
    probabilities = {}
    for classVal, classStat in stat.items():
        probabilities[classVal] = 1
        for i in range(len(classStat)):
            mean, stdev = classStat[i]
            x = test_vec[i]
            probabilities[classVal] *= calcGauss(x, mean, stdev)
    
    #выбор наибольшей вероятности и присваиваем соответствующий класс
    best_c, best_p = None, -1
    for classVal, probability in probabilities.items():
        if best_c is None or probability > best_p:
            best_p = probability
            best_c = classVal
    return best_c

class NBClassifier():
    def __init__(self):
        self.stat = dict()
    
    def fit(self, X_train):
        ln = len(X_train)
        #разделение по классам данных
        separated = {}
        for i in range(ln):
            vector = X_train[i]
            if (vector[-1] not in separated):
                separated[vector[-1]] = []
            separated[vector[-1]].append(vector)
            
        #медиана и стандартное отклонение для каждого параметра внутри каждого класса
        for classVal, instances in separated.items():
            m_d = [(mean(col), stdev(col)) for col in zip(*instances)]
            del m_d[-1]
            self.stat[classVal] = m_d
    
    def predict(self, test):
        predictions = []
        for i in range(len(test)):
            res = get_predict(self.stat, test[i])
            predictions.append(res)
        return predictions
    
start = time.time() 

nbc = NBClassifier()
nbc.fit(X_train[used_features].values)

test = X_test[used_features].values
y_pred = nbc.predict(test)
print('Неправильно предсказано: {} из {}\nТочность: {}%'.format((X_test["numdoors_cleaned"] != y_pred).sum(), len(y_pred), 100*(1-(X_test["numdoors_cleaned"] != y_pred).sum()/len(y_pred))))     
print('Матрица ошибок:\n', confusion_matrix(X_test["numdoors_cleaned"], y_pred))

finish = time.time()
print('Время работы собственного классификатора: {} секунд'.format(finish-start))

Разделено 201 строк на 134 обучающих и 67 тестовых строк
Неправильно предсказано: 6 из 67
Точность: 91.04477611940298%
Матрица ошибок:
 [[17  6]
 [ 0 44]]
Время работы собственного классификатора: 0.03127145767211914 секунд


## Получим те же результаты с помощью библиотеки sklearn

In [8]:
start = time.time()
gnb = GaussianNB()
used_features = [
    'aspiration_cleaned',
    'salon_cleaned',
    'wheels_cleaned']

gnb.fit(X_train[used_features].values,
       X_train['numdoors_cleaned'])
y_pred = gnb.predict(X_test[used_features])

print('Неправильно предсказано: {} из {}\nТочность: {}%'.format((X_test["numdoors_cleaned"] != y_pred).sum(), len(y_pred), 100*(1-(X_test["numdoors_cleaned"] != y_pred).sum()/len(y_pred))))     
print('Матрица ошибок:\n', confusion_matrix(X_test["numdoors_cleaned"], y_pred))

finish = time.time()
print('Время работы классификатора sklearn: {} секунд'.format(finish-start))

Неправильно предсказано: 6 из 67
Точность: 91.04477611940298%
Матрица ошибок:
 [[17  6]
 [ 0 44]]
Время работы классификатора sklearn: 0.015604972839355469 секунд


### Попробуем предсказать количество дверей, но теперь в зависимости от типа кузова (кабриолет, хэтчбек, седан и т.д.), размера двигателя и количества лошадиных сил.

In [9]:
start = time.time()

used_features = [
    'horsepower',
    'engine_size',
    'salon_cleaned',
    'numdoors_cleaned']
nbc = NBClassifier()
nbc.fit(X_train[used_features].values)

test = X_test[used_features].values
y_pred = nbc.predict(test)
print('Неправильно предсказано: {} из {}\nТочность: {}%'.format((X_test["numdoors_cleaned"] != y_pred).sum(), len(y_pred), 100*(1-(X_test["numdoors_cleaned"] != y_pred).sum()/len(y_pred))))     
print('Матрица ошибок:\n', confusion_matrix(X_test["numdoors_cleaned"], y_pred))

finish = time.time()
print('Время работы собственного классификатора: {} секунд'.format(finish-start))

Неправильно предсказано: 6 из 67
Точность: 91.04477611940298%
Матрица ошибок:
 [[20  3]
 [ 3 41]]
Время работы собственного классификатора: 0.03121352195739746 секунд


### С помощью библиотеки

In [17]:
start = time.time()
gnb = GaussianNB()
used_features = [
    'horsepower',
    'engine_size',
    'salon_cleaned']

gnb.fit(X_train[used_features].values,
       X_train['numdoors_cleaned'])
y_pred = gnb.predict(X_test[used_features])

print('Неправильно предсказано: {} из {}\nТочность: {}%'.format((X_test["numdoors_cleaned"] != y_pred).sum(), len(y_pred), 100*(1-(X_test["numdoors_cleaned"] != y_pred).sum()/len(y_pred))))     
print('Матрица ошибок:\n', confusion_matrix(X_test["numdoors_cleaned"], y_pred))

finish = time.time()
print('Время работы классификатора sklearn: {} секунд'.format(finish-start))

Неправильно предсказано: 6 из 67
Точность: 91.04477611940298%
Матрица ошибок:
 [[20  3]
 [ 3 41]]
Время работы классификатора sklearn: 0.015627145767211914 секунд


## Выводы


При работе с мультиномиальной моделью очень помогает скорректировать работу алгоритма сглаживание Лапласа, одна применение IDF параметра значительно повышает сложность и, соответственно, время работы алгоритма.
Можно было воспользоваться td idf методом из библиотеки,что я и попробовала, однако, опять таки, не смогла дождаться когда программа закончит обработку. 

Так же из-за ограничений по памяти и не совсем хорошей работы стеммера, возникли сложности с увеличением количества файлов. 

Время выполнения алгоритма моей реализации и библиотечной отличается во много раз, однако точность одинакова. 

Гауссовский наивный байес занял у меня намного больше времен, так как требует выполнения бОльшего количества операций. Мы предполагаем, что признаки возникают независимо (не всегда правда в реальной жизни) и, что признаки возникают с нормальным распределением, что тоже не соотвествует действительности. Однако результаты при этом оказываются с довольно хорошей точностью. 

Время работы моего Байеса и библиотечного различаются уже меньше, чем мультиномиальный. 

Из проделанной работы можно судить об простоте и быстроте работы НБА, тем не менее работающего с большой точностью.